In [10]:
##v5
import logging
from typing import Any, Dict, List, Optional, Sequence, Type, Union, Callable, Literal
from pydantic import BaseModel, Field
from langchain_core.tools import BaseTool
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.runnables import Runnable
from langchain_ibm import WatsonxLLM as BaseWatsonxLLM
from langchain_core.utils.function_calling import convert_to_openai_tool
from langchain_core.outputs import LLMResult, Generation, GenerationChunk
from langchain_core.messages import BaseMessage
from langchain_core.language_models import LanguageModelInput
import json

logger = logging.getLogger(__name__)

class WatsonxLLM(BaseWatsonxLLM):
    """Extended IBM watsonx.ai large language models."""

    bound_tools: Optional[List[BaseTool]] = Field(default=None, exclude=True)

    def __init__(self, *args, tools: Optional[List[BaseTool]] = None, **kwargs):
        super().__init__(*args, **kwargs)
        self.bound_tools = tools or []

    def _generate(
        self,
        prompts: List[str],
        stop: Optional[List[str]] = None,
        run_manager: Optional[Any] = None,
        stream: Optional[bool] = None,
        **kwargs: Any,
    ) -> LLMResult:
        """Call the IBM watsonx.ai inference endpoint which then generate the response.
        Args:
            prompts: List of strings (prompts) to pass into the model.
            stop: Optional list of stop words to use when generating.
            run_manager: Optional callback manager.
            stream: Optional boolean flag to indicate streaming generation.
        Returns:
            The full LLMResult output.
        """
        params = self._get_chat_params(stop=stop)
        should_stream = stream if stream is not None else self.streaming
        if should_stream:
            if len(prompts) > 1:
                raise ValueError(
                    f"WatsonxLLM currently only supports single prompt, got {prompts}"
                )
            generation = GenerationChunk(text="")
            stream_iter = self._stream(
                prompts[0], stop=stop, run_manager=run_manager, **kwargs
            )
            for chunk in stream_iter:
                if generation is None:
                    generation = chunk
                else:
                    generation += chunk
            assert generation is not None
            if isinstance(generation.generation_info, dict):
                llm_output = generation.generation_info.pop("llm_output")
                return LLMResult(generations=[[generation]], llm_output=llm_output)
            return LLMResult(generations=[[generation]])
        else:
            # Apply tools before generation


            if self.bound_tools:



                print("# Apply tools before generation")
                print("self.bound_tools", self.bound_tools)
                print("prompts[0]", prompts[0])

                tool_output = self._evaluate_tools(self.bound_tools, prompts[0])
                
                
                print("Tool output: %s", tool_output)
                
                logger.info("Tool output: %s", tool_output)
                # Include the tool output in the prompt
                system_prompt = (
                    f"You are an assistant with a tool. "
                    f"Provide a detailed answer to the user's query.\n\n"
                    f"User Query: {prompts[0]}\n\n"
                    f"Using the information below:\n"
                    f"Tool Results: {tool_output}\n\n"
                )
                prompts[0] = system_prompt
            print("prompts", prompts)
            response = self.watsonx_model.generate(
                prompt=prompts, params=params, **kwargs
            )
            return self._create_llm_result(response)

    def _evaluate_tools(self, tool_instances: List[BaseTool], input_text: str) -> str:
        """Evaluate the provided tools and return their combined output.
        Args:
            tool_instances: List of tools to evaluate.
            input_text: Input text to provide to the tools.
        Returns:
            Combined output of the tool evaluations.
        """
        print("At _evaluate_tools:")
        
        print("tool_instances", tool_instances)
        print("input_text", input_text)  

        combined_output = []
        for tool in tool_instances:
            try:
                result = tool.invoke(input_text)
                if isinstance(result, dict):
                    result = json.dumps(result, indent=2)  # Convert dict to formatted string
                elif isinstance(result, list):
                    result = "\n".join([json.dumps(item, indent=2) if isinstance(item, dict) else str(item) for item in result])  # Convert list to string
                combined_output.append(result)
            except Exception as e:
                logger.error(f"Error invoking tool {tool.name}: {str(e)}")
        return "\n\n".join(combined_output)

    @classmethod
    def bind_tools(
        cls,
        tools: Sequence[Union[Dict[str, Any], Type[BaseModel], Callable, BaseTool]],
        *,
        tool_choice: Optional[Union[Dict[str, str], Literal["any", "auto"], str]] = None,
        **kwargs: Any,
    ) -> 'WatsonxLLM':
        """Bind tool-like objects to this chat model.
        Assumes model is compatible with OpenAI tool-calling API.
        Args:
            tools: A list of tool definitions to bind to this chat model.
                Can be a dictionary, pydantic model, callable, or BaseTool. Pydantic
                models, callables, and BaseTools will be automatically converted to
                their schema dictionary representation.
            tool_choice: Which tool to require to call.
                Options are:
                    name of the tool (str): calls corresponding tool;
                    "auto" or None: automatically selects a tool (including no tool);
                    "any": force at least one tool to be called;
                    or a dict of the form:
                        {"type": "tool", "name": "tool_name"},
                        or {"type": "any"},
                        or {"type": "auto"};
            **kwargs: Any additional parameters to pass to the
                :class:`~langchain.runnable.Runnable` constructor.
        """
        formatted_tools = [convert_to_openai_tool(tool)["function"] for tool in tools]
        instance = cls(**kwargs)
        instance.bound_tools = tools
        if tool_choice is not None:
            kwargs["tool_choice"] = tool_choice
        return instance

    def _create_llm_result(self, response: List[dict]) -> LLMResult:
        """Create the LLMResult from the choices and prompts."""
        generations = []
        for res in response:
            results = res.get("results")
            if results:
                finish_reason = results[0].get("stop_reason")
                gen = Generation(
                    text=results[0].get("generated_text"),
                    generation_info={"finish_reason": finish_reason},
                )
                generations.append([gen])
        final_token_usage = self._extract_token_usage(response)
        llm_output = {
            "token_usage": final_token_usage,
            "model_id": self.model_id,
            "deployment_id": self.deployment_id,
        }
        return LLMResult(generations=generations, llm_output=llm_output)


In [11]:
from dotenv import load_dotenv
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
from ibm_watsonx_ai.foundation_models.utils.enums import DecodingMethods
import getpass
import os
load_dotenv()  # Load environment variables from .env file
def _set_env(var: str):
    load_dotenv()  # Load environment variables from .env file
    env_var = os.getenv(var)
    if not env_var:
        env_var = getpass.getpass(f"{var}: ")
        os.environ[var] = env_var
    return env_var

_set_env("TAVILY_API_KEY")
api_key = _set_env("WATSONX_API_KEY")
project_id = _set_env("PROJECT_ID")
url = "https://us-south.ml.cloud.ibm.com"
   
# Create an instance of WatsonxLLM
# WatsonxLLM initialization
parameters = {
    GenParams.DECODING_METHOD: DecodingMethods.SAMPLE.value,
    GenParams.MAX_NEW_TOKENS: 1000,
    GenParams.MIN_NEW_TOKENS: 50,
    GenParams.TEMPERATURE: 0.7,
    GenParams.TOP_K: 50,
    GenParams.TOP_P: 1
}
model_id = "ibm/granite-13b-instruct-v2"
watsonx_instance  = WatsonxLLM(
    model_id=model_id,
    url=url,
    apikey=api_key,
    project_id=project_id,
    params=parameters
)
watsonx_instance.invoke("How is the weather in Genova")    

prompts ['How is the weather in Genova']


'Genoa is located in Liguria, a region of Italy known for its mild climate. The average temperature in Genoa in January is 9.5°C, and the average temperature in July is 25.6°C. \n\n\nThe climate in Genoa is classified as humid subtropical, with hot, humid summers and mild, wet winters. Summers in Genoa are hot and humid, with average temperatures in the high 20s.'

In [12]:
# Example usage:
tool = TavilySearchResults(max_results=4)
llm_with_tools = watsonx_instance.bind_tools(tools=[tool],model_id=model_id,url=url,apikey=api_key,project_id=project_id,params=parameters) 
llm_with_tools.invoke("Who is Ruslan Magana?")  

# Apply tools before generation
self.bound_tools [TavilySearchResults(max_results=4)]
prompts[0] Who is Ruslan Magana?
At _evaluate_tools:
tool_instances [TavilySearchResults(max_results=4)]
input_text Who is Ruslan Magana?
Tool output: %s {
  "url": "https://ruslanmv.com/about",
  "content": "I'm Ruslan Magana Vsevolodovna. I'm a Data Scientist, a Cloud Architect and a Physicist. About me. I am Data Scientist specializing in Artificial Intelligence, with a distinct focus on Neural Networks. My core expertise lies in Generative AI and prompt engineering. I possess a strong commitment to precision and boast an extensive track ..."
}
{
  "url": "https://www.researchgate.net/profile/Ruslan-Magana-Vsevolodovna",
  "content": "Nov 2020. Ruslan Maga\u00f1a Vsevolodovna. The structure of odd-A Rh115,117 and Pd115,117 isotopes is studied by means of the neutron-proton interacting boson-fermion model (IBFM-2). JP=12+ quantum ..."
}
{
  "url": "https://scholar.google.com/citations?user=rWBrOpwAA

'Ruslan Magana Vsevolodovna is a Data Scientist, Cloud Architect, and Physicist. He has worked in the field of Artificial Intelligence, with a focus on Neural Networks. His work has been published in the International Journal of Modern Physics E, and he has contributed to open-source projects on GitHub.\n'

In [5]:
response = llm_with_tools._generate(prompts=["Who is Ruslan Magana?"])
print(response)

generations=[[Generation(text=' Ruslan Magana Vsevolodovna is a Data Scientist, Cloud Architect and Physicist. Ruslan Magana is theauthor of many publications, his most popular being "Transfer reactions between odd-odd and even-even nuclei by using the interacting boson Fermion model and $\\otionsless thanor equal to zero$ $\\otionsgreater thanor equal to zero$ decay in closure approximation$" and "Analysis of the one-neutron transfer reaction in 18O+76Se collisions at 275 MeV. Analysis of two-nucleon transfer reactions in the Ne20+Cd116 system at 306 MeV.$" You can learn more about Ruslan Magana on his website https://ruslanmv.com/about.', generation_info={'finish_reason': 'eos_token'})]] llm_output={'token_usage': {'generated_token_count': 149, 'input_token_count': 481}, 'model_id': 'ibm/granite-13b-instruct-v2', 'deployment_id': ''} run=None


In [37]:
#v5 fix
import logging
from typing import Any, Dict, List, Optional, Sequence, Type, Union, Callable, Literal
from pydantic import BaseModel, Field
from langchain_core.tools import BaseTool
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.runnables import Runnable
from langchain_ibm import WatsonxLLM as BaseWatsonxLLM
from langchain_core.utils.function_calling import convert_to_openai_tool
from langchain_core.outputs import LLMResult, Generation, GenerationChunk
from langchain_core.messages import BaseMessage
from langchain_core.language_models import LanguageModelInput
import json

logger = logging.getLogger(__name__)

class WatsonxLLM(BaseWatsonxLLM):
    """Extended IBM watsonx.ai large language models."""

    bound_tools: Optional[List[BaseTool]] = Field(default=None, exclude=True)

    def __init__(self, *args, tools: Optional[List[BaseTool]] = None, **kwargs):
        super().__init__(*args, **kwargs)
        self.bound_tools = tools or []

    def _generate(
        self,
        prompts: List[str],
        stop: Optional[List[str]] = None,
        run_manager: Optional[Any] = None,
        stream: Optional[bool] = None,
        **kwargs: Any,
    ) -> LLMResult:
        """Call the IBM watsonx.ai inference endpoint which then generate the response.
        Args:
            prompts: List of strings (prompts) to pass into the model.
            stop: Optional list of stop words to use when generating.
            run_manager: Optional callback manager.
            stream: Optional boolean flag to indicate streaming generation.
        Returns:
            The full LLMResult output.
        """
        params = self._get_chat_params(stop=stop)
        should_stream = stream if stream is not None else self.streaming
        if should_stream:
            if len(prompts) > 1:
                raise ValueError(
                    f"WatsonxLLM currently only supports single prompt, got {prompts}"
                )
            generation = GenerationChunk(text="")
            stream_iter = self._stream(
                prompts[0], stop=stop, run_manager=run_manager, **kwargs
            )
            for chunk in stream_iter:
                if generation is None:
                    generation = chunk
                else:
                    generation += chunk
            assert generation is not None
            if isinstance(generation.generation_info, dict):
                llm_output = generation.generation_info.pop("llm_output")
                return LLMResult(generations=[[generation]], llm_output=llm_output)
            return LLMResult(generations=[[generation]])
        else:
            # Apply tools before generation
            if self.bound_tools:
                print("# Apply tools before generation")
                print("self.bound_tools", self.bound_tools)
                print("prompts[0]", prompts[0])


                tool_output = self._evaluate_tools(self.bound_tools, prompts[0])
                logger.info("Tool output: %s", tool_output)
                print("Tool output: %s", tool_output)
                # Include the tool output in the prompt
                system_prompt = (
                    f"You are an assistant with a tool. "
                    f"Provide a detailed answer to the user's query.\n\n"
                    f"User Query: {prompts[0]}\n\n"
                    f"Using the information below:\n"
                    f"Tool Results: {tool_output}\n\n"
                )
                prompts[0] = system_prompt
            print("prompts", prompts)
            response = self.watsonx_model.generate(
                prompt=prompts, params=params, **kwargs
            )
            return self._create_llm_result(response)

    def _evaluate_tools(self, tool_instances: List[BaseTool], input_text: str) -> str:
        """Evaluate the provided tools and return their combined output.
        Args:
            tool_instances: List of tools to evaluate.
            input_text: Input text to provide to the tools.
        Returns:
            Combined output of the tool evaluations.
        """
        print("At _evaluate_tools:")
        
        print("tool_instances", tool_instances)
        print("input_text", input_text)


        # Extract the actual user query from the input_text from the prompt of agent
        user_query = input_text.split("Human:")[-1].strip()
        print("Extracted user query:", user_query)
        print("type query:", type(user_query))
        user_query='Who is Ruslan Magana'
        combined_output = []
        for tool in tool_instances:
            try:
                result = tool.invoke(user_query) #input_text
                if isinstance(result, dict):
                    #isinstance 1
                    print("isinstance 1",result)
                    result = json.dumps(result, indent=2)  # Convert dict to formatted string
                elif isinstance(result, list):
                    #isinstance 2
                    print("isinstance 2",result)
                    result = "\n".join([json.dumps(item, indent=2) if isinstance(item, dict) else str(item) for item in result])  # Convert list to string
                combined_output.append(result)
            except Exception as e:
                logger.error(f"Error invoking tool {tool.name}: {str(e)}")
        return "\n\n".join(combined_output)

    @classmethod
    def bind_tools(
        cls,
        tools: Sequence[Union[Dict[str, Any], Type[BaseModel], Callable, BaseTool]],
        *,
        tool_choice: Optional[Union[Dict[str, str], Literal["any", "auto"], str]] = None,
        **kwargs: Any,
    ) -> 'WatsonxLLM':
        """Bind tool-like objects to this chat model.
        Assumes model is compatible with OpenAI tool-calling API.
        Args:
            tools: A list of tool definitions to bind to this chat model.
                Can be a dictionary, pydantic model, callable, or BaseTool. Pydantic
                models, callables, and BaseTools will be automatically converted to
                their schema dictionary representation.
            tool_choice: Which tool to require to call.
                Options are:
                    name of the tool (str): calls corresponding tool;
                    "auto" or None: automatically selects a tool (including no tool);
                    "any": force at least one tool to be called;
                    or a dict of the form:
                        {"type": "tool", "name": "tool_name"},
                        or {"type": "any"},
                        or {"type": "auto"};
            **kwargs: Any additional parameters to pass to the
                :class:`~langchain.runnable.Runnable` constructor.
        """
        formatted_tools = [convert_to_openai_tool(tool)["function"] for tool in tools]
        instance = cls(**kwargs)
        instance.bound_tools = tools
        if tool_choice is not None:
            kwargs["tool_choice"] = tool_choice
        return instance

    def _create_llm_result(self, response: List[dict]) -> LLMResult:
        """Create the LLMResult from the choices and prompts."""
        generations = []
        for res in response:
            results = res.get("results")
            if results:
                finish_reason = results[0].get("stop_reason")
                gen = Generation(
                    text=results[0].get("generated_text"),
                    generation_info={"finish_reason": finish_reason},
                )
                generations.append([gen])
        final_token_usage = self._extract_token_usage(response)
        llm_output = {
            "token_usage": final_token_usage,
            "model_id": self.model_id,
            "deployment_id": self.deployment_id,
        }
        return LLMResult(generations=generations, llm_output=llm_output)


In [34]:
from dotenv import load_dotenv
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
from ibm_watsonx_ai.foundation_models.utils.enums import DecodingMethods
import getpass
import os
load_dotenv()  # Load environment variables from .env file
def _set_env(var: str):
    load_dotenv()  # Load environment variables from .env file
    env_var = os.getenv(var)
    if not env_var:
        env_var = getpass.getpass(f"{var}: ")
        os.environ[var] = env_var
    return env_var

_set_env("TAVILY_API_KEY")
api_key = _set_env("WATSONX_API_KEY")
project_id = _set_env("PROJECT_ID")
url = "https://us-south.ml.cloud.ibm.com"
   
# Create an instance of WatsonxLLM
# WatsonxLLM initialization
parameters = {
    GenParams.DECODING_METHOD: DecodingMethods.SAMPLE.value,
    GenParams.MAX_NEW_TOKENS: 1000,
    GenParams.MIN_NEW_TOKENS: 50,
    GenParams.TEMPERATURE: 0.7,
    GenParams.TOP_K: 50,
    GenParams.TOP_P: 1
}
model_id = "ibm/granite-13b-instruct-v2"
watsonx_instance  = WatsonxLLM(
    model_id=model_id,
    url=url,
    apikey=api_key,
    project_id=project_id,
    params=parameters
)
#watsonx_instance.invoke("How is the weather in Genova")    

In [35]:
# Example usage:
tool = TavilySearchResults(max_results=4)
llm_with_tools = watsonx_instance.bind_tools(tools=[tool],model_id=model_id,url=url,apikey=api_key,project_id=project_id,params=parameters) 
llm_with_tools.invoke("Who is Ruslan Magana?")  

# Apply tools before generation
self.bound_tools [TavilySearchResults(max_results=4)]
prompts[0] Who is Ruslan Magana?
At _evaluate_tools:
tool_instances [TavilySearchResults(max_results=4)]
input_text Who is Ruslan Magana?
Extracted user query: Who is Ruslan Magana?
type query: <class 'str'>
isinstance 2 [{'url': 'https://ruslanmv.com/about', 'content': "I'm Ruslan Magana Vsevolodovna. I'm a Data Scientist, a Cloud Architect and a Physicist. About me. I am Data Scientist specializing in Artificial Intelligence, with a distinct focus on Neural Networks. My core expertise lies in Generative AI and prompt engineering. I possess a strong commitment to precision and boast an extensive track ..."}, {'url': 'https://www.researchgate.net/profile/Ruslan-Magana-Vsevolodovna', 'content': 'Nov 2020. Ruslan Magaña Vsevolodovna. The structure of odd-A Rh115,117 and Pd115,117 isotopes is studied by means of the neutron-proton interacting boson-fermion model (IBFM-2). JP=12+ quantum ...'}, {'url': 'h

'Ruslan Magana is a Data Scientist, Cloud Architect and Physicist. Ruslan Magana expertise is in Generative AI and prompt engineering. Ruslan Magana is also involved in Watsonx.ai. Ruslan Magana is a PhD.'

# Apply tools before generation
self.bound_tools [TavilySearchResults(max_results=4)]
prompts[0] Who is Ruslan Magana?
At _evaluate_tools:
tool_instances [TavilySearchResults(max_results=4)]
input_text Who is Ruslan Magana?
isinstance 2 [{'url': 'https://ruslanmv.com/about', 'content': "I'm Ruslan Magana Vsevolodovna. I'm a Data Scientist, a Cloud Architect and a Physicist. About me. I am Data Scientist specializing in Artificial Intelligence, with a distinct focus on Neural Networks. My core expertise lies in Generative AI and prompt engineering. I possess a strong commitment to precision and boast an extensive track ..."}, {'url': 'https://www.researchgate.net/profile/Ruslan-Magana-Vsevolodovna', 'content': 'Nov 2020. Ruslan Magaña Vsevolodovna. The structure of odd-A Rh115,117 and Pd115,117 isotopes is studied by means of the neutron-proton interacting boson-fermion model (IBFM-2). JP=12+ quantum ...'}, {'url': 'https://scholar.google.com/citations?user=rWBrOpwAAAAJ', 'content': 'Ru

'Ruslan Magana Vsevolodovna is a Data Scientist, Cloud Architect and Physicist. He specializes in Artificial Intelligence, with a distinct focus on Neural Networks. He is the founder of ruslanmv.com and has a strong commitment to precision.'

In [36]:
from langchain_core.messages import BaseMessage, ToolMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.graph import END, StateGraph
from langchain_community.tools.tavily_search import TavilySearchResults
from dotenv import load_dotenv
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
from ibm_watsonx_ai.foundation_models.utils.enums import DecodingMethods
import getpass
import os
def create_agent(llm, tools, system_message: str, params_dict):
    """Create an agent."""
    parameters = params_dict["parameters"]
    model_id = params_dict["model_id"]
    url = params_dict["url"]
    api_key = params_dict["api_key"]
    project_id = params_dict["project_id"]

    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "You are a helpful AI assistant, collaborating with other assistants."
                " Use the provided tools to progress towards answering the question."
                " If you are unable to fully answer, that's OK, another assistant with different tools "
                " will help where you left off. Execute what you can to make progress."
                " If you or any of the other assistants have the final answer or deliverable,"
                " prefix your response with FINAL ANSWER so the team knows to stop."
                " You have access to the following tools: {tool_names}.\n{system_message}",
            ),
            MessagesPlaceholder(variable_name="messages"),
        ]
    )

    prompt = prompt.partial(system_message=system_message)
    prompt = prompt.partial(tool_names=", ".join([tool.name for tool in tools]))

    llm_with_tools = llm.bind_tools(
        tools=tools,
        model_id=model_id,
        url=url,
        apikey=api_key,
        project_id=project_id,
        params=parameters
    )

    return prompt | llm_with_tools

# Example usage
params_dict = {
    "parameters": parameters,
    "model_id": model_id,
    "url": url,
    "api_key": api_key,
    "project_id": project_id
}

tavily_tool = TavilySearchResults(max_results=4)

research_agent = create_agent(
    watsonx_instance,
    [tavily_tool],
    system_message="You should provide accurate data for the chart_generator to use.",
    params_dict=params_dict,
)

# Invoke the agent
response = research_agent.invoke(["Who is Ruslan Magana?"])
print(response)


# Apply tools before generation
self.bound_tools [TavilySearchResults(max_results=4)]
prompts[0] System: You are a helpful AI assistant, collaborating with other assistants. Use the provided tools to progress towards answering the question. If you are unable to fully answer, that's OK, another assistant with different tools  will help where you left off. Execute what you can to make progress. If you or any of the other assistants have the final answer or deliverable, prefix your response with FINAL ANSWER so the team knows to stop. You have access to the following tools: tavily_search_results_json.
You should provide accurate data for the chart_generator to use.
Human: Who is Ruslan Magana?
At _evaluate_tools:
tool_instances [TavilySearchResults(max_results=4)]
input_text System: You are a helpful AI assistant, collaborating with other assistants. Use the provided tools to progress towards answering the question. If you are unable to fully answer, that's OK, another assistant with diff

# Apply tools before generation
self.bound_tools [TavilySearchResults(max_results=4)]
prompts[0] System: You are a helpful AI assistant, collaborating with other assistants. Use the provided tools to progress towards answering the question. If you are unable to fully answer, that's OK, another assistant with different tools  will help where you left off. Execute what you can to make progress. If you or any of the other assistants have the final answer or deliverable, prefix your response with FINAL ANSWER so the team knows to stop. You have access to the following tools: tavily_search_results_json.
You should provide accurate data for the chart_generator to use.
Human: Who is Ruslan Magana?
At _evaluate_tools:
tool_instances [TavilySearchResults(max_results=4)]
input_text System: You are a helpful AI assistant, collaborating with other assistants. Use the provided tools to progress towards answering the question. If you are unable to fully answer, that's OK, another assistant with diff

In [24]:
# Example usage:
tool = TavilySearchResults(max_results=4)
llm_with_tools = watsonx_instance.bind_tools(tools=[tool],model_id=model_id,url=url,apikey=api_key,project_id=project_id,params=parameters) 
llm_with_tools.invoke("Who is Ruslan Magana?")  

# Apply tools before generation
self.bound_tools [TavilySearchResults(max_results=4)]
prompts[0] Who is Ruslan Magana?
At _evaluate_tools:
tool_instances [TavilySearchResults(max_results=4)]
input_text Who is Ruslan Magana?
Extracted user query: Who is Ruslan Magana?
isinstance 2 [{'url': 'https://ruslanmv.com/about', 'content': "I'm Ruslan Magana Vsevolodovna. I'm a Data Scientist, a Cloud Architect and a Physicist. About me. I am Data Scientist specializing in Artificial Intelligence, with a distinct focus on Neural Networks. My core expertise lies in Generative AI and prompt engineering. I possess a strong commitment to precision and boast an extensive track ..."}, {'url': 'https://www.researchgate.net/profile/Ruslan-Magana-Vsevolodovna', 'content': 'Nov 2020. Ruslan Magaña Vsevolodovna. The structure of odd-A Rh115,117 and Pd115,117 isotopes is studied by means of the neutron-proton interacting boson-fermion model (IBFM-2). JP=12+ quantum ...'}, {'url': 'https://scholar.google.com/

'Ruslan Magana Vsevolodovna is a Ph.D. in Physics, who is specialized in Artificial Intelligence and Machine Learning. He is also the founder of Ruslan MV, where he offers his consultancy services in the field of Data Science and Machine Learning.'